In [113]:
import pandas as pd
import numpy as np
from sqlalchemy import text as sa_text, select
from sqlalchemy.orm import sessionmaker, Session
from sqlalchemy import text as sa_text, Connection, create_engine, NullPool
from sqlalchemy.ext.asyncio import create_async_engine, AsyncSession
from sqlalchemy.orm import sessionmaker, Session
import os

In [114]:
PG_USR = os.getenv('POSTGRES_USER', 'username')
PG_PWD = os.getenv('POSTGRES_PASSWORD', 'password')
PG_HOST = os.getenv('POSTGRES_HOST', 'localhost')
PG_PORT = os.getenv('POSTGRES_PORT', '5432')
PG_DB_NAME = os.getenv('POSTGRES_DB', 'postgres')
db = create_engine(f'postgresql://{PG_USR}:{PG_PWD}@{PG_HOST}:{PG_PORT}/{PG_DB_NAME}') 

In [115]:
query = sa_text(
    """
    select * from counter_consumer_events 
    """
)

events = pd.read_sql(query, db)
events

,id_x,obj_consumer_id,counter_event_created,gcal_in_system,gcal_out_system,subset,leak,supply_temp,return_temp,heat_thermal_energy,errors,errors_desc,index,id_y,description,days_of_work,event_created,event_closed,id
0,NaN,1952,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,101894.0,NaN,Температура в квартире ниже нормативной,0.0,2024-05-09 15:15:07+00:00,2024-05-09 16:19:03.147000+00:00,40562.0
1,NaN,1952,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,101938.0,NaN,Температура в квартире ниже нормативной,0.0,2024-05-09 15:31:40+00:00,2024-05-09 16:21:44.410000+00:00,40563.0
2,NaN,1952,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,103750.0,NaN,Температура в квартире ниже нормативной,0.0,2024-05-11 11:08:03+00:00,2024-05-11 11:24:44.481000+00:00,40565.0
3,NaN,1952,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,104297.0,NaN,Температура в квартире ниже нормативной,0.0,2024-05-12 11:34:49+00:00,2024-05-12 16:11:19.522000+00:00,40561.0
4,NaN,1952,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,104521.0,NaN,Температура в квартире ниже нормативной,0.0,2024-05-12 17:59:13+00:00,2024-05-12 19:13:39.773000+00:00,40564.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1027914,NaN,1952,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,73975.0,NaN,Температура в квартире ниже нормативной,0.0,2024-01-07 07:09:09+00:00,2024-01-07 08:22:44.252000+00:00,40555.0
1027915,NaN,1952,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,76082.0,NaN,Температура в квартире ниже нормативной,0.0,2024-01-09 15:13:17+00:00,2024-01-09 18:44:59.085000+00:00,40556.0
1027916,NaN,1952,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,77969.0,NaN,Температура в квартире ниже нормативной,0.0,2024-01-12 14:13:34+00:00,2024-01-12 15:23:20.596000+00:00,40557.0
1027917,NaN,1952,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,100509.0,NaN,T1 < min,0.0,2024-04-16 11:34:59+00:00,2024-04-16 11:43:25.884000+00:00,40559.0


In [116]:
def set_date(x):
    if not pd.isnull(x['counter_event_created']):
        return x['counter_event_created']
    if not pd.isnull(x['event_created']):
        return x['event_created']
    if not pd.isnull(x['event_closed']):
        return x['event_closed']

events['time'] = events.apply(lambda x: set_date(x), axis=1)

In [117]:
events = events.sort_values(by='time', ascending=True)
events

,id_x,obj_consumer_id,counter_event_created,gcal_in_system,gcal_out_system,subset,leak,supply_temp,return_temp,heat_thermal_energy,errors,errors_desc,index,id_y,description,days_of_work,event_created,event_closed,id,time
831591,178741.0,469,2023-01-10 00:00:00+00:00,0.000000,12.628906,12.628906,0.000000,25.477112,24.542732,0.000000,U,Отсутствие электропитания,NaN,NaN,None,NaN,NaT,NaT,NaN,2023-01-10 00:00:00+00:00
80390,559892.0,2692,2023-01-10 00:00:00+00:00,0.000000,0.000000,0.000000,0.000000,20.970274,21.338549,0.000000,0,na,NaN,NaN,None,NaN,NaT,NaT,NaN,2023-01-10 00:00:00+00:00
161050,662430.0,3316,2023-01-10 00:00:00+00:00,0.000000,0.000000,0.000000,0.000000,26.364866,26.209284,0.000000,0,na,NaN,NaN,None,NaN,NaT,NaT,NaN,2023-01-10 00:00:00+00:00
982367,376390.0,1630,2023-01-10 00:00:00+00:00,0.000000,0.000000,0.000000,0.000000,20.160000,21.190000,0.000000,"D,E",Разница температур в подающем и обратном трубо...,NaN,NaN,None,NaN,NaT,NaT,NaN,2023-01-10 00:00:00+00:00
720784,471976.0,2213,2023-01-10 00:00:00+00:00,0.000000,0.670000,0.670000,0.000000,37.133333,33.558333,0.000000,g,Расход меньше минимального,NaN,NaN,None,NaN,NaT,NaT,NaN,2023-01-10 00:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69618,542339.0,2594,2024-12-04 00:00:00+00:00,297.730469,297.095703,0.000000,0.634766,50.991959,42.348549,2.544800,U,Отсутствие электропитания,NaN,NaN,None,NaN,NaT,NaT,NaN,2024-12-04 00:00:00+00:00
695431,746900.0,3855,2024-12-04 00:00:00+00:00,100.356445,100.247070,0.000000,0.109375,50.997219,42.635006,0.840821,0,na,NaN,NaN,None,NaN,NaT,NaT,NaN,2024-12-04 00:00:00+00:00
281078,821954.0,4327,2024-12-04 00:00:00+00:00,57.324707,55.939453,0.000000,1.385254,71.071175,45.890194,1.443954,0,na,NaN,NaN,None,NaN,NaT,NaT,NaN,2024-12-04 00:00:00+00:00
688629,350290.0,1480,2024-12-04 00:00:00+00:00,1376.343750,1373.921875,0.000000,2.421875,45.299179,41.217026,5.631714,U,Отсутствие электропитания,NaN,NaN,None,NaN,NaT,NaT,NaN,2024-12-04 00:00:00+00:00


In [118]:
# events.drop(columns=['id_y', 'id_x', 'index', 'id'], inplace=True, axis=1)
events.reset_index()
events

,id_x,obj_consumer_id,counter_event_created,gcal_in_system,gcal_out_system,subset,leak,supply_temp,return_temp,heat_thermal_energy,errors,errors_desc,index,id_y,description,days_of_work,event_created,event_closed,id,time
831591,178741.0,469,2023-01-10 00:00:00+00:00,0.000000,12.628906,12.628906,0.000000,25.477112,24.542732,0.000000,U,Отсутствие электропитания,NaN,NaN,None,NaN,NaT,NaT,NaN,2023-01-10 00:00:00+00:00
80390,559892.0,2692,2023-01-10 00:00:00+00:00,0.000000,0.000000,0.000000,0.000000,20.970274,21.338549,0.000000,0,na,NaN,NaN,None,NaN,NaT,NaT,NaN,2023-01-10 00:00:00+00:00
161050,662430.0,3316,2023-01-10 00:00:00+00:00,0.000000,0.000000,0.000000,0.000000,26.364866,26.209284,0.000000,0,na,NaN,NaN,None,NaN,NaT,NaT,NaN,2023-01-10 00:00:00+00:00
982367,376390.0,1630,2023-01-10 00:00:00+00:00,0.000000,0.000000,0.000000,0.000000,20.160000,21.190000,0.000000,"D,E",Разница температур в подающем и обратном трубо...,NaN,NaN,None,NaN,NaT,NaT,NaN,2023-01-10 00:00:00+00:00
720784,471976.0,2213,2023-01-10 00:00:00+00:00,0.000000,0.670000,0.670000,0.000000,37.133333,33.558333,0.000000,g,Расход меньше минимального,NaN,NaN,None,NaN,NaT,NaT,NaN,2023-01-10 00:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69618,542339.0,2594,2024-12-04 00:00:00+00:00,297.730469,297.095703,0.000000,0.634766,50.991959,42.348549,2.544800,U,Отсутствие электропитания,NaN,NaN,None,NaN,NaT,NaT,NaN,2024-12-04 00:00:00+00:00
695431,746900.0,3855,2024-12-04 00:00:00+00:00,100.356445,100.247070,0.000000,0.109375,50.997219,42.635006,0.840821,0,na,NaN,NaN,None,NaN,NaT,NaT,NaN,2024-12-04 00:00:00+00:00
281078,821954.0,4327,2024-12-04 00:00:00+00:00,57.324707,55.939453,0.000000,1.385254,71.071175,45.890194,1.443954,0,na,NaN,NaN,None,NaN,NaT,NaT,NaN,2024-12-04 00:00:00+00:00
688629,350290.0,1480,2024-12-04 00:00:00+00:00,1376.343750,1373.921875,0.000000,2.421875,45.299179,41.217026,5.631714,U,Отсутствие электропитания,NaN,NaN,None,NaN,NaT,NaT,NaN,2024-12-04 00:00:00+00:00


In [119]:

query = sa_text(
    """
    select * from public.event_types 
    """
)

events_classes = pd.read_sql(query, db)
events_classes = events_classes.set_index('event_name').T.to_dict('index').get('id')
keys = list(events_classes.keys())
keys

['P1 <= 0',
 'P2 <= 0',
 'T1 > max',
 'T1 < min',
 'Авария',
 'Недостаточная температура подачи ЦО (Недотоп)',
 'Превышение температуры подачи ЦО (Перетоп)',
 'Утечка теплоносителя',
 'Течь в системе отопления',
 'Температура в квартире ниже нормативной',
 'Крупные пожары',
 'Температура в помещении общего пользования ниже нормативной',
 'Аварийная протечка труб в подъезде',
 'Протечка труб в подъезде',
 'Отсутствие отопления в доме',
 'Сильная течь в системе отопления']

In [120]:
def set_event_class(x, events_classes: dict, keys: list):
    if x in keys:
        return events_classes.get(x)
    # return 1
    return 0

events['event_class'] = events['description'].apply(lambda x: set_event_class(x, events_classes, keys))
events

,id_x,obj_consumer_id,counter_event_created,gcal_in_system,gcal_out_system,subset,leak,supply_temp,return_temp,heat_thermal_energy,...,errors_desc,index,id_y,description,days_of_work,event_created,event_closed,id,time,event_class
831591,178741.0,469,2023-01-10 00:00:00+00:00,0.000000,12.628906,12.628906,0.000000,25.477112,24.542732,0.000000,...,Отсутствие электропитания,NaN,NaN,None,NaN,NaT,NaT,NaN,2023-01-10 00:00:00+00:00,0
80390,559892.0,2692,2023-01-10 00:00:00+00:00,0.000000,0.000000,0.000000,0.000000,20.970274,21.338549,0.000000,...,na,NaN,NaN,None,NaN,NaT,NaT,NaN,2023-01-10 00:00:00+00:00,0
161050,662430.0,3316,2023-01-10 00:00:00+00:00,0.000000,0.000000,0.000000,0.000000,26.364866,26.209284,0.000000,...,na,NaN,NaN,None,NaN,NaT,NaT,NaN,2023-01-10 00:00:00+00:00,0
982367,376390.0,1630,2023-01-10 00:00:00+00:00,0.000000,0.000000,0.000000,0.000000,20.160000,21.190000,0.000000,...,Разница температур в подающем и обратном трубо...,NaN,NaN,None,NaN,NaT,NaT,NaN,2023-01-10 00:00:00+00:00,0
720784,471976.0,2213,2023-01-10 00:00:00+00:00,0.000000,0.670000,0.670000,0.000000,37.133333,33.558333,0.000000,...,Расход меньше минимального,NaN,NaN,None,NaN,NaT,NaT,NaN,2023-01-10 00:00:00+00:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69618,542339.0,2594,2024-12-04 00:00:00+00:00,297.730469,297.095703,0.000000,0.634766,50.991959,42.348549,2.544800,...,Отсутствие электропитания,NaN,NaN,None,NaN,NaT,NaT,NaN,2024-12-04 00:00:00+00:00,0
695431,746900.0,3855,2024-12-04 00:00:00+00:00,100.356445,100.247070,0.000000,0.109375,50.997219,42.635006,0.840821,...,na,NaN,NaN,None,NaN,NaT,NaT,NaN,2024-12-04 00:00:00+00:00,0
281078,821954.0,4327,2024-12-04 00:00:00+00:00,57.324707,55.939453,0.000000,1.385254,71.071175,45.890194,1.443954,...,na,NaN,NaN,None,NaN,NaT,NaT,NaN,2024-12-04 00:00:00+00:00,0
688629,350290.0,1480,2024-12-04 00:00:00+00:00,1376.343750,1373.921875,0.000000,2.421875,45.299179,41.217026,5.631714,...,Отсутствие электропитания,NaN,NaN,None,NaN,NaT,NaT,NaN,2024-12-04 00:00:00+00:00,0


In [121]:
events.drop(columns=['id_y', 'id_x', 'index', 'id', 'description', 'counter_event_created', 'event_created', 'event_closed', 'errors_desc'], inplace=True, axis=1)
events

,obj_consumer_id,gcal_in_system,gcal_out_system,subset,leak,supply_temp,return_temp,heat_thermal_energy,errors,days_of_work,time,event_class
831591,469,0.000000,12.628906,12.628906,0.000000,25.477112,24.542732,0.000000,U,NaN,2023-01-10 00:00:00+00:00,0
80390,2692,0.000000,0.000000,0.000000,0.000000,20.970274,21.338549,0.000000,0,NaN,2023-01-10 00:00:00+00:00,0
161050,3316,0.000000,0.000000,0.000000,0.000000,26.364866,26.209284,0.000000,0,NaN,2023-01-10 00:00:00+00:00,0
982367,1630,0.000000,0.000000,0.000000,0.000000,20.160000,21.190000,0.000000,"D,E",NaN,2023-01-10 00:00:00+00:00,0
720784,2213,0.000000,0.670000,0.670000,0.000000,37.133333,33.558333,0.000000,g,NaN,2023-01-10 00:00:00+00:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...
69618,2594,297.730469,297.095703,0.000000,0.634766,50.991959,42.348549,2.544800,U,NaN,2024-12-04 00:00:00+00:00,0
695431,3855,100.356445,100.247070,0.000000,0.109375,50.997219,42.635006,0.840821,0,NaN,2024-12-04 00:00:00+00:00,0
281078,4327,57.324707,55.939453,0.000000,1.385254,71.071175,45.890194,1.443954,0,NaN,2024-12-04 00:00:00+00:00,0
688629,1480,1376.343750,1373.921875,0.000000,2.421875,45.299179,41.217026,5.631714,U,NaN,2024-12-04 00:00:00+00:00,0


In [122]:
events['year'] = events['time'].dt.year
events['month'] = events['time'].dt.month
events['season'] = events['month'] % 12 // 3 + 1
events['day'] = events['time'].dt.day
# df['work_time'] = (df['event_closed'] - df['event_created']).dt.days
events['day_of_week'] = events['time'].dt.dayofweek
events['is_weekend'] = events['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)
events

,obj_consumer_id,gcal_in_system,gcal_out_system,subset,leak,supply_temp,return_temp,heat_thermal_energy,errors,days_of_work,time,event_class,year,month,season,day,day_of_week,is_weekend
831591,469,0.000000,12.628906,12.628906,0.000000,25.477112,24.542732,0.000000,U,NaN,2023-01-10 00:00:00+00:00,0,2023,1,1,10,1,0
80390,2692,0.000000,0.000000,0.000000,0.000000,20.970274,21.338549,0.000000,0,NaN,2023-01-10 00:00:00+00:00,0,2023,1,1,10,1,0
161050,3316,0.000000,0.000000,0.000000,0.000000,26.364866,26.209284,0.000000,0,NaN,2023-01-10 00:00:00+00:00,0,2023,1,1,10,1,0
982367,1630,0.000000,0.000000,0.000000,0.000000,20.160000,21.190000,0.000000,"D,E",NaN,2023-01-10 00:00:00+00:00,0,2023,1,1,10,1,0
720784,2213,0.000000,0.670000,0.670000,0.000000,37.133333,33.558333,0.000000,g,NaN,2023-01-10 00:00:00+00:00,0,2023,1,1,10,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69618,2594,297.730469,297.095703,0.000000,0.634766,50.991959,42.348549,2.544800,U,NaN,2024-12-04 00:00:00+00:00,0,2024,12,1,4,2,0
695431,3855,100.356445,100.247070,0.000000,0.109375,50.997219,42.635006,0.840821,0,NaN,2024-12-04 00:00:00+00:00,0,2024,12,1,4,2,0
281078,4327,57.324707,55.939453,0.000000,1.385254,71.071175,45.890194,1.443954,0,NaN,2024-12-04 00:00:00+00:00,0,2024,12,1,4,2,0
688629,1480,1376.343750,1373.921875,0.000000,2.421875,45.299179,41.217026,5.631714,U,NaN,2024-12-04 00:00:00+00:00,0,2024,12,1,4,2,0


In [123]:
events

,obj_consumer_id,gcal_in_system,gcal_out_system,subset,leak,supply_temp,return_temp,heat_thermal_energy,errors,days_of_work,time,event_class,year,month,season,day,day_of_week,is_weekend
831591,469,0.000000,12.628906,12.628906,0.000000,25.477112,24.542732,0.000000,U,NaN,2023-01-10 00:00:00+00:00,0,2023,1,1,10,1,0
80390,2692,0.000000,0.000000,0.000000,0.000000,20.970274,21.338549,0.000000,0,NaN,2023-01-10 00:00:00+00:00,0,2023,1,1,10,1,0
161050,3316,0.000000,0.000000,0.000000,0.000000,26.364866,26.209284,0.000000,0,NaN,2023-01-10 00:00:00+00:00,0,2023,1,1,10,1,0
982367,1630,0.000000,0.000000,0.000000,0.000000,20.160000,21.190000,0.000000,"D,E",NaN,2023-01-10 00:00:00+00:00,0,2023,1,1,10,1,0
720784,2213,0.000000,0.670000,0.670000,0.000000,37.133333,33.558333,0.000000,g,NaN,2023-01-10 00:00:00+00:00,0,2023,1,1,10,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69618,2594,297.730469,297.095703,0.000000,0.634766,50.991959,42.348549,2.544800,U,NaN,2024-12-04 00:00:00+00:00,0,2024,12,1,4,2,0
695431,3855,100.356445,100.247070,0.000000,0.109375,50.997219,42.635006,0.840821,0,NaN,2024-12-04 00:00:00+00:00,0,2024,12,1,4,2,0
281078,4327,57.324707,55.939453,0.000000,1.385254,71.071175,45.890194,1.443954,0,NaN,2024-12-04 00:00:00+00:00,0,2024,12,1,4,2,0
688629,1480,1376.343750,1373.921875,0.000000,2.421875,45.299179,41.217026,5.631714,U,NaN,2024-12-04 00:00:00+00:00,0,2024,12,1,4,2,0


In [124]:
from datetime import datetime

print((datetime.now() - datetime.now()).days)

-1


In [125]:
## сматчить предыдущий инц
# events['event_id'] = events['accident']
consumers = events['obj_consumer_id'].unique().tolist()
events['last_event_class'] = 0
events['last_event_days'] = 0

# df['between_created'] = 0
train_df = pd.DataFrame()
# last = 0
for consumer in consumers:
    consumers_df = events[events['obj_consumer_id'] == consumer].sort_values(by='time', ascending=True)
    consumers_df = consumers_df.reset_index()
    last_event_class = 0
    last_event_day = 0
    for i, row in consumers_df.iterrows():
        if i > 0:
            if consumers_df.at[i - 1, 'event_class'] != 0:
                last_event_class = consumers_df.at[i - 1, 'event_class']
                last_event_day = consumers_df.at[i, 'time']
            # abs((df['event_closed'] - df['event_created']).dt.days.astype(float))
            consumers_df.at[i, 'last_event_class'] = last_event_class
            if last_event_day != 0:
                consumers_df.at[i, 'last_event_days'] = (consumers_df.at[i, 'time'] - last_event_day).days
        else:
            consumers_df.at[i, 'last_event_days'] = 0
            consumers_df.at[i, 'last_event_class'] = 0
            # consumers_df.at[i, 'last_event_class'] = last
    # consumers_df = consumers_df[(consumers_df['event_class'] != 0) | (consumers_df['last_event_class'] != 0)]
    train_df = pd.concat([train_df, consumers_df])
train_df = train_df.sort_values(by=['obj_consumer_id', 'time'], ascending=(True, False))

In [126]:
# train_df[train_df['obj_consumer_id'] == 2][['time', 'event_class', 'last_event_class', 'last_event_days']]
train_df.fillna({
    'gcal_in_system': 0,
    'gcal_out_system': 0,
    'subset': 0,
    'leak': 0,
    'supply_temp': 0,
    'return_temp': 0,
    'heat_thermal_energy': 0,
    'days_of_work': 0,
    'errors': 0,
}, inplace=True)
# train_df['gcal_in_system'].fillna(0, inplace=True)
train_df

,index,obj_consumer_id,gcal_in_system,gcal_out_system,subset,leak,supply_temp,return_temp,heat_thermal_energy,errors,...,time,event_class,year,month,season,day,day_of_week,is_weekend,last_event_class,last_event_days
215,781280,2,101.741211,99.705078,0.0,2.036133,67.891945,46.142269,2.214905,0,...,2024-12-04 00:00:00+00:00,0,2024,12,1,4,2,0,3,291
214,781279,2,105.114258,102.973145,0.0,2.141113,81.523445,50.317730,3.284791,0,...,2024-12-03 00:00:00+00:00,0,2024,12,1,3,1,0,3,290
213,781278,2,137.442871,134.461914,0.0,2.980957,105.358482,65.716919,5.472442,0,...,2024-12-02 00:00:00+00:00,0,2024,12,1,2,0,0,3,289
212,781277,2,106.800965,104.517701,0.0,2.283264,110.550529,60.279949,5.393513,0,...,2024-12-01 00:00:00+00:00,0,2024,12,1,1,6,1,3,288
211,781276,2,103.018555,100.957031,0.0,2.061524,66.854248,46.293636,2.120330,0,...,2024-11-04 00:00:00+00:00,0,2024,11,4,4,0,0,3,261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,430046,5575,227.445312,223.656250,0.0,3.789062,57.036049,49.115894,1.799477,0,...,2023-02-11 00:00:00+00:00,0,2023,2,1,11,5,1,0,0
3,430045,5575,0.001465,0.000000,0.0,0.001465,18.734449,18.525652,0.000000,D,...,2023-02-10 00:00:00+00:00,0,2023,2,1,10,4,0,0,0
2,430044,5575,228.365235,225.150390,0.0,3.214845,66.732010,55.752815,2.498787,D,...,2023-01-12 00:00:00+00:00,0,2023,1,1,12,3,0,0,0
1,430043,5575,227.490235,223.740234,0.0,3.750001,57.059227,49.222752,1.780853,0,...,2023-01-11 00:00:00+00:00,0,2023,1,1,11,2,0,0,0


In [127]:
train_df['errors'] = train_df['errors'].apply(lambda x:len(x.split(',')) if x != 0 else 0)
train_df

,index,obj_consumer_id,gcal_in_system,gcal_out_system,subset,leak,supply_temp,return_temp,heat_thermal_energy,errors,...,time,event_class,year,month,season,day,day_of_week,is_weekend,last_event_class,last_event_days
215,781280,2,101.741211,99.705078,0.0,2.036133,67.891945,46.142269,2.214905,1,...,2024-12-04 00:00:00+00:00,0,2024,12,1,4,2,0,3,291
214,781279,2,105.114258,102.973145,0.0,2.141113,81.523445,50.317730,3.284791,1,...,2024-12-03 00:00:00+00:00,0,2024,12,1,3,1,0,3,290
213,781278,2,137.442871,134.461914,0.0,2.980957,105.358482,65.716919,5.472442,1,...,2024-12-02 00:00:00+00:00,0,2024,12,1,2,0,0,3,289
212,781277,2,106.800965,104.517701,0.0,2.283264,110.550529,60.279949,5.393513,1,...,2024-12-01 00:00:00+00:00,0,2024,12,1,1,6,1,3,288
211,781276,2,103.018555,100.957031,0.0,2.061524,66.854248,46.293636,2.120330,1,...,2024-11-04 00:00:00+00:00,0,2024,11,4,4,0,0,3,261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,430046,5575,227.445312,223.656250,0.0,3.789062,57.036049,49.115894,1.799477,1,...,2023-02-11 00:00:00+00:00,0,2023,2,1,11,5,1,0,0
3,430045,5575,0.001465,0.000000,0.0,0.001465,18.734449,18.525652,0.000000,1,...,2023-02-10 00:00:00+00:00,0,2023,2,1,10,4,0,0,0
2,430044,5575,228.365235,225.150390,0.0,3.214845,66.732010,55.752815,2.498787,1,...,2023-01-12 00:00:00+00:00,0,2023,1,1,12,3,0,0,0
1,430043,5575,227.490235,223.740234,0.0,3.750001,57.059227,49.222752,1.780853,1,...,2023-01-11 00:00:00+00:00,0,2023,1,1,11,2,0,0,0


In [128]:
train_df.drop('index', inplace=True, axis=1)
train_df

,obj_consumer_id,gcal_in_system,gcal_out_system,subset,leak,supply_temp,return_temp,heat_thermal_energy,errors,days_of_work,time,event_class,year,month,season,day,day_of_week,is_weekend,last_event_class,last_event_days
215,2,101.741211,99.705078,0.0,2.036133,67.891945,46.142269,2.214905,1,0.0,2024-12-04 00:00:00+00:00,0,2024,12,1,4,2,0,3,291
214,2,105.114258,102.973145,0.0,2.141113,81.523445,50.317730,3.284791,1,0.0,2024-12-03 00:00:00+00:00,0,2024,12,1,3,1,0,3,290
213,2,137.442871,134.461914,0.0,2.980957,105.358482,65.716919,5.472442,1,0.0,2024-12-02 00:00:00+00:00,0,2024,12,1,2,0,0,3,289
212,2,106.800965,104.517701,0.0,2.283264,110.550529,60.279949,5.393513,1,0.0,2024-12-01 00:00:00+00:00,0,2024,12,1,1,6,1,3,288
211,2,103.018555,100.957031,0.0,2.061524,66.854248,46.293636,2.120330,1,0.0,2024-11-04 00:00:00+00:00,0,2024,11,4,4,0,0,3,261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,5575,227.445312,223.656250,0.0,3.789062,57.036049,49.115894,1.799477,1,0.0,2023-02-11 00:00:00+00:00,0,2023,2,1,11,5,1,0,0
3,5575,0.001465,0.000000,0.0,0.001465,18.734449,18.525652,0.000000,1,0.0,2023-02-10 00:00:00+00:00,0,2023,2,1,10,4,0,0,0
2,5575,228.365235,225.150390,0.0,3.214845,66.732010,55.752815,2.498787,1,0.0,2023-01-12 00:00:00+00:00,0,2023,1,1,12,3,0,0,0
1,5575,227.490235,223.740234,0.0,3.750001,57.059227,49.222752,1.780853,1,0.0,2023-01-11 00:00:00+00:00,0,2023,1,1,11,2,0,0,0


In [129]:
from sklearn import preprocessing

# преобразует строковые параметры в числа 
# Exampe: "Hello" "Hello" "world" [0,0,1]
class MultiColumnLabelEncoder:
    def __init__(self, columns=None):
        self.columns = columns  # array of column names to encode

    def fit(self, X, y=None):
        return self  # not relevant here

    def transform(self, X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = preprocessing.LabelEncoder().fit_transform(output[col])
        else:
            for col_name, col in output.iteritems():
                output[col_name] = preprocessing.LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)

In [130]:
query = sa_text(
    """
    select
        oss.id obj_source_satation_id,
        oss.e_power, oss.t_power,
        oss.boiler_count, oss.turbine_count,
        -- oss.launched_date,
        oc.obj_consumer_station_id,
        oc.id obj_consumer_id,
        oc.load_gvs, oc.load_fact, oc.heat_load, oc.vent_load,
        oc.total_area,
        oc.location_district_id, oc.location_area_id,
        -- oc.address,
        oc.street, oc.house_number,
        -- oc.corpus_number, oc.soor_type, oc.soor_number,
        oc.b_class, oc.floors,
        oc.wear_pct,
        oc.build_year,
        oc.sock_type,
        oc.energy_class,
        oc.operating_mode,
        oc.priority,
        oc.is_dispatch
     from public.obj_consumers oc
     join public.obj_consumer_stations cs on oc.obj_consumer_station_id = cs.id
     join obj_source_consumer_stations ocs on oc.obj_consumer_station_id = ocs.obj_consumer_station_id
     join public.obj_source_stations oss on oss.id = ocs.obj_source_station_id
    """
)
consumers = pd.read_sql(query, db)
consumers

,obj_source_satation_id,e_power,t_power,boiler_count,turbine_count,obj_consumer_station_id,obj_consumer_id,load_gvs,load_fact,heat_load,...,house_number,b_class,floors,wear_pct,build_year,sock_type,energy_class,operating_mode,priority,is_dispatch
0,1,1420,3709,23,8,1,2,0.1442,0.3474,0.1678,...,37,жилые,12,0.0,0,МКД,Нет данных,Круглосуточно,7,True
1,1,1420,3709,23,8,1,3,0.17,0.3765623,0.379,...,41,жилые,5,0.0,0,МКД,Нет данных,Круглосуточно,7,True
2,1,1420,3709,23,8,1,4,0.09,0.198,0.207,...,41,жилые,5,0.0,0,МКД,Нет данных,Круглосуточно,7,True
3,1,1420,3709,23,8,1,8,0.16,0.352,0.364,...,41,жилые,5,0.0,0,МКД,Нет данных,Круглосуточно,7,True
4,1,1420,3709,23,8,1,9,0.17,0.374,0.364,...,41,жилые,5,0.0,0,МКД,Нет данных,Круглосуточно,7,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5504,26,1070,3402,22,11,5312,5312,0.090571,0.3169985,0.503,...,66Д,нежилые,4,10.0,2014,Социальный,C,Круглосуточно,3,False
5505,1,1420,3709,23,8,669,670,0.05,0.175,0.09,...,13,нежилые,2,0.0,0,Социальный,Нет данных,9:00 – 18:00,3,False
5506,1,1420,3709,23,8,1199,1199,0.0,0.0,0.0,...,16,нежилые,3,0.0,0,Социальный,Нет данных,9:00 – 18:00,3,False
5507,1,1420,3709,23,8,1392,1392,0.04587,0.160545,0.1935,...,5А,нежилые,3,12.0,2008,Социальный,A++,9:00 – 18:00,3,False


In [131]:
# Преобразуем в числовые параметры
consumers_df = MultiColumnLabelEncoder(columns=[
    'street', 'house_number',
    'b_class', 'sock_type', 'energy_class',
    'operating_mode', 
]).fit_transform(consumers)


In [132]:
len(consumers_df['obj_consumer_id'].unique().tolist())

5509

In [249]:
all_train_df = pd.merge(consumers_df, train_df, how='left', on='obj_consumer_id')
all_train_df

,obj_source_satation_id,e_power,t_power,boiler_count,turbine_count,obj_consumer_station_id,obj_consumer_id,load_gvs,load_fact,heat_load,...,time,event_class,year,month,season,day,day_of_week,is_weekend,last_event_class,last_event_days
0,1,1420,3709,23,8,1,2,0.1442,0.3474,0.1678,...,2024-12-04 00:00:00+00:00,0.0,2024.0,12.0,1.0,4.0,2.0,0.0,3.0,291.0
1,1,1420,3709,23,8,1,2,0.1442,0.3474,0.1678,...,2024-12-03 00:00:00+00:00,0.0,2024.0,12.0,1.0,3.0,1.0,0.0,3.0,290.0
2,1,1420,3709,23,8,1,2,0.1442,0.3474,0.1678,...,2024-12-02 00:00:00+00:00,0.0,2024.0,12.0,1.0,2.0,0.0,0.0,3.0,289.0
3,1,1420,3709,23,8,1,2,0.1442,0.3474,0.1678,...,2024-12-01 00:00:00+00:00,0.0,2024.0,12.0,1.0,1.0,6.0,1.0,3.0,288.0
4,1,1420,3709,23,8,1,2,0.1442,0.3474,0.1678,...,2024-11-04 00:00:00+00:00,0.0,2024.0,11.0,4.0,4.0,0.0,0.0,3.0,261.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1029088,1,1420,3709,23,8,4391,4391,0.19,0.418,0.36,...,2023-02-11 00:00:00+00:00,0.0,2023.0,2.0,1.0,11.0,5.0,1.0,0.0,0.0
1029089,1,1420,3709,23,8,4391,4391,0.19,0.418,0.36,...,2023-02-10 00:00:00+00:00,0.0,2023.0,2.0,1.0,10.0,4.0,0.0,0.0,0.0
1029090,1,1420,3709,23,8,4391,4391,0.19,0.418,0.36,...,2023-01-12 00:00:00+00:00,0.0,2023.0,1.0,1.0,12.0,3.0,0.0,0.0,0.0
1029091,1,1420,3709,23,8,4391,4391,0.19,0.418,0.36,...,2023-01-11 00:00:00+00:00,0.0,2023.0,1.0,1.0,11.0,2.0,0.0,0.0,0.0


In [250]:
all_train_df= all_train_df[['obj_source_satation_id',
 'e_power',
 't_power',
 'boiler_count',
 'turbine_count',
 'obj_consumer_station_id',
 'obj_consumer_id',
 'load_gvs',
 'load_fact',
 'heat_load',
 'vent_load',
 'total_area',
 'location_district_id',
 'location_area_id',
 'street',
 'house_number',
 'b_class',
 'floors',
 'wear_pct',
 'build_year',
 'sock_type',
 'energy_class',
 'operating_mode',
 'priority',
 'is_dispatch',
 'gcal_in_system',
 'gcal_out_system',
 'gcal_out_system',
 'subset',
 'leak',
 'supply_temp',
 'return_temp',
 'heat_thermal_energy',
 'errors',
 'days_of_work',
 'time',
 'year',
 'month',
 'season',
 'day',
 'day_of_week',
 'is_weekend',
 'last_event_class',
 'last_event_days',
 'event_class',]]
# len(all_train_df['obj_consumer_id'].unique().tolist())
all_train_df

,obj_source_satation_id,e_power,t_power,boiler_count,turbine_count,obj_consumer_station_id,obj_consumer_id,load_gvs,load_fact,heat_load,...,time,year,month,season,day,day_of_week,is_weekend,last_event_class,last_event_days,event_class
0,1,1420,3709,23,8,1,2,0.1442,0.3474,0.1678,...,2024-12-04 00:00:00+00:00,2024.0,12.0,1.0,4.0,2.0,0.0,3.0,291.0,0.0
1,1,1420,3709,23,8,1,2,0.1442,0.3474,0.1678,...,2024-12-03 00:00:00+00:00,2024.0,12.0,1.0,3.0,1.0,0.0,3.0,290.0,0.0
2,1,1420,3709,23,8,1,2,0.1442,0.3474,0.1678,...,2024-12-02 00:00:00+00:00,2024.0,12.0,1.0,2.0,0.0,0.0,3.0,289.0,0.0
3,1,1420,3709,23,8,1,2,0.1442,0.3474,0.1678,...,2024-12-01 00:00:00+00:00,2024.0,12.0,1.0,1.0,6.0,1.0,3.0,288.0,0.0
4,1,1420,3709,23,8,1,2,0.1442,0.3474,0.1678,...,2024-11-04 00:00:00+00:00,2024.0,11.0,4.0,4.0,0.0,0.0,3.0,261.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1029088,1,1420,3709,23,8,4391,4391,0.19,0.418,0.36,...,2023-02-11 00:00:00+00:00,2023.0,2.0,1.0,11.0,5.0,1.0,0.0,0.0,0.0
1029089,1,1420,3709,23,8,4391,4391,0.19,0.418,0.36,...,2023-02-10 00:00:00+00:00,2023.0,2.0,1.0,10.0,4.0,0.0,0.0,0.0,0.0
1029090,1,1420,3709,23,8,4391,4391,0.19,0.418,0.36,...,2023-01-12 00:00:00+00:00,2023.0,1.0,1.0,12.0,3.0,0.0,0.0,0.0,0.0
1029091,1,1420,3709,23,8,4391,4391,0.19,0.418,0.36,...,2023-01-11 00:00:00+00:00,2023.0,1.0,1.0,11.0,2.0,0.0,0.0,0.0,0.0


In [135]:
df_with_out = all_train_df[all_train_df['last_event_class'].notna()]

In [136]:
# df_with_out.drop('time', inplace=True, axis=1)

In [137]:
df_with_out

,obj_source_satation_id,e_power,t_power,boiler_count,turbine_count,obj_consumer_station_id,obj_consumer_id,load_gvs,load_fact,heat_load,...,time,event_class,year,month,season,day,day_of_week,is_weekend,last_event_class,last_event_days
0,1,1420,3709,23,8,1,2,0.1442,0.3474,0.1678,...,2024-12-04 00:00:00+00:00,0.0,2024.0,12.0,1.0,4.0,2.0,0.0,3.0,291.0
1,1,1420,3709,23,8,1,2,0.1442,0.3474,0.1678,...,2024-12-03 00:00:00+00:00,0.0,2024.0,12.0,1.0,3.0,1.0,0.0,3.0,290.0
2,1,1420,3709,23,8,1,2,0.1442,0.3474,0.1678,...,2024-12-02 00:00:00+00:00,0.0,2024.0,12.0,1.0,2.0,0.0,0.0,3.0,289.0
3,1,1420,3709,23,8,1,2,0.1442,0.3474,0.1678,...,2024-12-01 00:00:00+00:00,0.0,2024.0,12.0,1.0,1.0,6.0,1.0,3.0,288.0
4,1,1420,3709,23,8,1,2,0.1442,0.3474,0.1678,...,2024-11-04 00:00:00+00:00,0.0,2024.0,11.0,4.0,4.0,0.0,0.0,3.0,261.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1029088,1,1420,3709,23,8,4391,4391,0.19,0.418,0.36,...,2023-02-11 00:00:00+00:00,0.0,2023.0,2.0,1.0,11.0,5.0,1.0,0.0,0.0
1029089,1,1420,3709,23,8,4391,4391,0.19,0.418,0.36,...,2023-02-10 00:00:00+00:00,0.0,2023.0,2.0,1.0,10.0,4.0,0.0,0.0,0.0
1029090,1,1420,3709,23,8,4391,4391,0.19,0.418,0.36,...,2023-01-12 00:00:00+00:00,0.0,2023.0,1.0,1.0,12.0,3.0,0.0,0.0,0.0
1029091,1,1420,3709,23,8,4391,4391,0.19,0.418,0.36,...,2023-01-11 00:00:00+00:00,0.0,2023.0,1.0,1.0,11.0,2.0,0.0,0.0,0.0


In [138]:
# df_with_out.to_csv('train.csv', index=False)

In [245]:
def split_date(events: pd.DataFrame):
    events['year'] = events['time'].dt.year
    events['month'] = events['time'].dt.month
    events['season'] = events['month'] % 12 // 3 + 1
    events['day'] = events['time'].dt.day
    # df['work_time'] = (df['event_closed'] - df['event_created']).dt.days
    events['day_of_week'] = events['time'].dt.dayofweek
    events['is_weekend'] = events['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)
    return events

ss = all_train_df.groupby(['obj_consumer_id']).first()
ss = ss.reset_index()
# ss[['obj_consumer_id', 'time']]
f = ss[ss['event_class'] != 0]['event_class'].count()
ss['time'] = datetime.now()
ss['time'] = ss['time'].astype('datetime64[ns]')
# ss['time'] = ss['time'].apply(lambda x: x + pd.Timedelta("1 day"))
ss = split_date(ss)
# ss['time']

ss['last_event_days'] = ss.apply(lambda x: x['last_event_days'] if x['event_class'] == 0 else 0, axis=1)
ss['last_event_class'] = ss.apply(lambda x: x['event_class'], axis=1)
ss['event_class'] = 0
# ss = ss[ss['last_event_class'].isnull()]
# ss.fillna({
#     'gcal_in_system': ss['gcal_in_system'].mean(),
#     'gcal_out_system': ss['gcal_out_system'].mean(),
#     'subset': ss['subset'].mean(),
#     'leak': ss['leak'].mean(),
#     'supply_temp': ss['supply_temp'].mean(),
#     'return_temp': ss['return_temp'].mean(),
# }, inplace=True)
# ss = ss.sample(frac=1).reset_index(drop=True)
ss = ss.sort_values(by=['leak'], ascending=True, na_position='last')
ss.interpolate(method='linear', inplace=True)
# ss.infer_objects(copy=True).interpolate(method='spline', inplace=True)
# ss = ss[ss['last_event_class'].isnull()]
# ss.fillna({
#     'gcal_in_system': ss['gcal_in_system'].mean(),
#     'gcal_out_system': ss['gcal_out_system'].mean(),
#     'subset': ss['subset'].mean(),
#     'leak': ss['leak'].mean(),
#     'supply_temp': ss['supply_temp'].mean(),
#     'return_temp': ss['return_temp'].mean(),
# }, inplace=True)
# ss[(ss['obj_consumer_id'] == 1) | (ss['obj_consumer_id'] == 10)][['obj_consumer_id', 'gcal_in_system', 'gcal_out_system', 'subset', 'leak', 'supply_temp', 'return_temp']]
# ss[['obj_consumer_id', 'gcal_in_system', 'gcal_out_system', 'subset', 'leak', 'supply_temp', 'return_temp']]
# 225.2 222.3 1.5
ss.isnull().sum()
ss = ss.sort_values(by=['obj_consumer_id'], ascending=True)

/var/folders/qz/qth62xx942x93nyp4rrsdt_w0000gn/T/ipykernel_46209/244292520.py:35: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  ss.interpolate(method='linear', inplace=True)


,obj_consumer_id,obj_source_satation_id,e_power,t_power,boiler_count,turbine_count,obj_consumer_station_id,load_gvs,load_fact,heat_load,...,time,event_class,year,month,season,day,day_of_week,is_weekend,last_event_class,last_event_days
0,1,1,1420,3709,23,8,1,0.01,0.035,0.105,...,2024-06-13 14:13:56.851996,0,2024,6,3,13,3,0,0.0,0.0
1,2,1,1420,3709,23,8,1,0.1442,0.3474,0.1678,...,2024-06-13 14:13:56.851996,0,2024,6,3,13,3,0,0.0,291.0
2,3,1,1420,3709,23,8,1,0.17,0.3765623,0.379,...,2024-06-13 14:13:56.851996,0,2024,6,3,13,3,0,0.0,251.0
3,4,1,1420,3709,23,8,1,0.09,0.198,0.207,...,2024-06-13 14:13:56.851996,0,2024,6,3,13,3,0,0.0,295.0
4,5,1,1420,3709,23,8,1,0.13912,0.333364,0.2854,...,2024-06-13 14:13:56.851996,0,2024,6,3,13,3,0,0.0,186.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5504,5571,148,330,868,7,3,5571,0.09,0.19865,0.17,...,2024-06-13 14:13:56.851996,0,2024,6,3,13,3,0,0.0,0.0
5505,5572,1,1420,3709,23,8,5572,0.2,0.447956,0.454,...,2024-06-13 14:13:56.851996,0,2024,6,3,13,3,0,0.0,266.0
5506,5573,1,1420,3709,23,8,5572,0.0,0.0,0.15,...,2024-06-13 14:13:56.851996,0,2024,6,3,13,3,0,0.0,410.0
5507,5574,1,1420,3709,23,8,5572,0.112219,0.2504321,0.247551,...,2024-06-13 14:13:56.851996,0,2024,6,3,13,3,0,0.0,378.0


In [169]:


# def set_event_class_for_predict():
    


,time,new_time
0,NaT,NaT
1,2024-12-04 00:00:00+00:00,2024-12-05 00:00:00+00:00
2,2024-12-04 00:00:00+00:00,2024-12-05 00:00:00+00:00
3,2024-12-04 00:00:00+00:00,2024-12-05 00:00:00+00:00
4,2024-12-04 00:00:00+00:00,2024-12-05 00:00:00+00:00
...,...,...
5504,2024-12-04 00:00:00+00:00,2024-12-05 00:00:00+00:00
5505,2024-12-04 00:00:00+00:00,2024-12-05 00:00:00+00:00
5506,2024-12-04 00:00:00+00:00,2024-12-05 00:00:00+00:00
5507,2024-12-04 00:00:00+00:00,2024-12-05 00:00:00+00:00


In [156]:
df_with_out[df_with_out['obj_consumer_id'] == 5575][['obj_consumer_id', 'time']]

,obj_consumer_id,time
679587,5575,2024-12-04 00:00:00+00:00
679588,5575,2024-12-03 00:00:00+00:00
679589,5575,2024-12-02 00:00:00+00:00
679590,5575,2024-12-01 00:00:00+00:00
679591,5575,2024-11-04 00:00:00+00:00
...,...,...
679807,5575,2023-02-11 00:00:00+00:00
679808,5575,2023-02-10 00:00:00+00:00
679809,5575,2023-01-12 00:00:00+00:00
679810,5575,2023-01-11 00:00:00+00:00
